In [29]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_pickle('../Data/cleaned_data_completed.pickle')

In [31]:
df.columns

Index(['definition', 'expert', 'dataset', 'scores', 'F1', 'sexist_F1',
       'non_sexist_F1'],
      dtype='object')

In [32]:
def split_results(name):
    list_of_datasets = ['CallMeSexist','EDOS', 'RedditGuest','EXIST', 'hatecheck' ]
    CMSB = []
    EDOS = []
    REDDIT = []
    EXIST = []
    HC = []
    for definition, expert, dataset, F1 in zip(df['definition'],df['expert'],df['dataset'],df['F1']):
        if definition == name:
            if dataset == list_of_datasets[0]:
                CMSB.append(F1)
            if dataset == list_of_datasets[1]:
                EDOS.append(F1)
            if dataset == list_of_datasets[2]:
                REDDIT.append(F1)
            if dataset == list_of_datasets[3]:
                EXIST.append(F1)
            if dataset == list_of_datasets[4]:
                HC.append(F1)

    return   CMSB, EDOS, REDDIT, EXIST, HC 



In [33]:
CMSB, EDOS, REDDIT, EXIST, HC = split_results(name='GPT')


CORRELATION WITH LENGTH


In [34]:
df_len = pd.read_csv('../../PartIII_definitions/Data/all_definitions.csv')

df_len.columns

Index(['ResponseId', 'ParticipantDefine', 'ChosenDefinition',
       'GeneratedDefinition'],
      dtype='object')

In [35]:
human = df_len['ParticipantDefine']
generated = df_len['GeneratedDefinition']
hybrid = df_len['ChosenDefinition']

def get_len(lista):
    list_of_length = []
    for i,l in enumerate(lista):
        if i == 6 or i == 7:
            pass
        else:
            lungezza = l.split()
            list_of_length.append(len(lungezza))


    return list_of_length

human_len = get_len(human)
hybrid_len = get_len(hybrid)
generated_len = get_len(generated)


In [36]:

definitionz = ['GPT', 'hybrid', 'participant']

for defi in definitionz:
    CMSB, EDOS, REDDIT, EXIST, HC = split_results(name=defi)
    combined_list = [CMSB, EDOS, REDDIT, EXIST, HC]
    name_of_lists =['CMSB', 'EDOS', 'REDDIT', 'EXIST', 'HC']
    
    if defi == 'GPT':
        lengths = generated_len
    if defi == 'hybrid':
        lengths = hybrid_len
    if defi == 'participant':
        lengths = human_len
    
    for i, lista in enumerate(combined_list):
        correlation = np.corrcoef(lengths, lista)[0, 1]
        formatted_corr = f"{correlation:.2f}"
        
        # Remove leading zero only for positive numbers
        if formatted_corr.startswith("0"):
            formatted_corr = formatted_corr[1:]
        elif formatted_corr.startswith("-0"):
            formatted_corr = "-" + formatted_corr[2:]

        print(f"Correlation between definition {defi} and {name_of_lists[i]}")
        print(formatted_corr)





Correlation between definition GPT and CMSB
-.67
Correlation between definition GPT and EDOS
-.38
Correlation between definition GPT and REDDIT
-.15
Correlation between definition GPT and EXIST
.18
Correlation between definition GPT and HC
-.25
Correlation between definition hybrid and CMSB
-.48
Correlation between definition hybrid and EDOS
-.05
Correlation between definition hybrid and REDDIT
-.38
Correlation between definition hybrid and EXIST
-.14
Correlation between definition hybrid and HC
-.61
Correlation between definition participant and CMSB
-.85
Correlation between definition participant and EDOS
-.62
Correlation between definition participant and REDDIT
-.65
Correlation between definition participant and EXIST
-.85
Correlation between definition participant and HC
-.24


CORRELATION WITH RATING


In [37]:
df_likert = pd.read_csv('/Users/matteo/Desktop/GESIS/MIM/Code/Definitions_Interaction_ds.csv')

first= df_likert['Likert_happy_definit_1']
second= df_likert['Likert_happy_definit_2']
third = df_likert['Likert_happy_definit_3']
fourth = df_likert['Likert_happy_definit_4']
fifth = df_likert['Likert_happy_definit_5']
sixth = df_likert['Likert_happy_definit_6']

sixth_new =[]
for f in sixth:
    rs = 8 - f
    sixth_new.append(rs)


sum_of_rates = [sum(x) for x in zip(first, second,third,fourth,fifth,sixth_new)]

del sum_of_rates[7]
del sum_of_rates[6]


In [38]:

CMSB, EDOS, REDDIT, EXIST, HC = split_results(name='hybrid')
combined_list = [CMSB, EDOS, REDDIT, EXIST, HC]
name_of_lists =['CMSB', 'EDOS', 'REDDIT', 'EXIST', 'HC']


for i,lista in enumerate(combined_list):
    correlation = np.corrcoef(sum_of_rates, lista)[0, 1]
    print(f"Correlation between definition rating of hybrid definition on {name_of_lists[i]}")
    print(f"{correlation:.2f}")

Correlation between definition rating of hybrid definition on CMSB
0.28
Correlation between definition rating of hybrid definition on EDOS
0.12
Correlation between definition rating of hybrid definition on REDDIT
0.20
Correlation between definition rating of hybrid definition on EXIST
-0.35
Correlation between definition rating of hybrid definition on HC
-0.33


CORRELATION WITH COSINE SIMILARITY

In [39]:
import matplotlib.pyplot as plt
import seaborn as sns
import inspect
from sentence_transformers import SentenceTransformer
import numpy as np


In [40]:
experts = ['EXP_1', 'EXP_2', 'EXP_3', 'EXP_4', 'EXP_5', 'EXP_6', 'EXP_9', 'EXP_10', 'EXP_11']
defi_part = ['Sexism is discrimination based on gender, including stereotypes and prejudice. It can take many forms, from offensive language to sexual violence. ', 'I would define sexism as a way to imply stereotypical or unrespectful opinion on gender. ', 'Believing that one sex is superior to another. It includes different forms of harmful stereotyping including benevolent sexism, etc.', 'A prescriptive set of behaviors or qualities, that women (and men) are supposed to exhibit to conform to traditional gender roles.  \n', 'Abusive language targeted to an individual or group because of their sex and/or gender.', 'Statements or behaviours that reinforce the idea that men are given more power and status than women, that men and women are (or should be) distinguished on the basis of their social roles and characteristics; and relationships between men and women are defined by sexual reproduction and intimacy. Sexism can take the form of hostile and benevolent sexism.','Sexism is linked to beliefs around the fundamental nature of women and men and the roles they should play in society. Sexist assumptions about women and men, which manifest themselves as gender stereotypes, can rank one gender as superior to another.', "Broadly, sexism is prejudice or discrimination based on one's sex or gender - a form of oppression that results primarily in disadvantages for women and queer people.\nA more technical definition of sexism is a system of norms, beliefs, and practices that normalise, perpetuate, and legitimise a binary, heteronormative, and/or deterministic framework of understanding sex and gender. Sexism can manifest in many different ways and it is deeply embedded in our institutions, practices, traditions, cultural norms and values. So detecting and understanding how sexism works necessarily involves studying how society is structured, organised, and governed.", "A view of an identity that is defined by social beliefs but is not necessarily supported by a ground truth or doesn't reflect their actual abilities"]
defi_hybrid = ["Sexism refers to prejudice, discrimination, or stereotyping based on a person's sex or gender, typically with a bias against one gender over the other. It can manifest in various forms, including unequal treatment, unfair expectations, and harmful attitudes or beliefs about individuals based on their sex. Examples of everyday sexism in the workplace could include paying women less than men for the same job or assuming certain roles are only suitable for one gender. Online sexism could involve making derogatory comments based on gender or perpetuating harmful stereotypes in digital spaces. ", "Sexism is a form of prejudice, stereotyping, or discrimination based on gender, often targeting women. This bias reflects the belief in one gender's superiority, resulting in unfair treatment, negative stereotypes, and limited opportunities solely due to gender. For example, a common illustration of sexism is the gender pay gap, where women are paid less than men for the same work. Sexism can manifest in different forms like unequal treatment, unfair stereotypes, or restrictions on opportunities, impacting both individuals and society. If you require more examples or details for each form of sexism, feel free to ask! Sexism's effects can be harmful, leading to inequalities, perpetuating stereotypes, and limiting opportunities for individuals. It primarily affects women but can also impact men in different contexts. Resolving sexism involves efforts at individual, societal, and institutional levels through education, promoting gender equality, implementing fair policies, and challenging discriminatory practices. If you need further clarification or details on this topic, please let me know!", "Sexism refers to discrimination, prejudice, or stereotyping based on a person's gender. It can manifest in various forms, such as treating individuals differently because of their gender, assuming certain characteristics or behaviors based on gender, or denying opportunities to someone because of their gender. Sexism can affect people of all genders, but it is often directed towards women and can have harmful effects on their lives and opportunities. There are different types of sexism, such as hostile sexism, benevolent sexism, and ambivalent sexism.", "It's fascinating to see your expertise in discerning subtle linguistic cues to identify sexism in texts. The definition of sexism centers on discrimination, prejudice, stereotyping, and biased behaviors toward individuals based on gender, leading to unequal treatment and marginalization. Evaluating texts for sexism entails examining portrayals of gender superiority or inferiority, reinforcement of stereotypes, and gender-based limitations, which involves analyzing the text's structure, tone, and phrasing to ascertain if it perpetuates gender inequality. ", 'Sexism specifically refers to discrimination, prejudice, or stereotyping directed towards individuals or groups based on their gender, where gender is the underlying cause of the abusive behavior. Mentioning the gender of the target alone may not be sufficient for a sentence to be considered sexist. Your task is to help identify and address harmful and unfair attitudes and behaviors towards people because of their gender, while also recognizing that counter speech to challenge these harmful attitudes is permissible.', "Hostile sexism refers to overtly negative and hostile attitudes and behaviors towards women based on the belief that women are inferior to men. This can involve open discrimination, derogatory remarks, and devaluation of women. Benevolent sexism, on the other hand, involves attitudes and behaviors that may appear positive or chivalrous on the surface, but actually serve to reinforce traditional gender roles and keep women in subordinate positions. These attitudes often arise from the societal norms and expectations that distinguish men and women based on their perceived social roles and characteristics. Examples of benevolent sexism include patronizing behavior, protective gestures, or idealizing women as pure and delicate. To detect sexism in text, look for language or scenarios that suggest unequal treatment, devaluation, or reinforcement of traditional gender roles. Pay attention to how women are portrayed, whether they are being reduced to stereotypes, denied agency, or discriminated against. Both hostile and benevolent sexism can manifest in various ways in written content, so it's important to analyze the overall context and underlying messages being conveyed.", "Sexism is a form of discrimination, prejudice, or stereotyping based on a person's sex or gender, typically towards women or girls. It involves treating people differently or unfairly because of their gender, and can be expressed in various forms, such as unequal treatment, stereotypes, or systemic sexism within societal structures.", 'Sexism is the manifestation of discriminatory, prejudiced, or stereotypical attitudes towards individuals based on their sex or gender. Understanding sexism includes recognizing and addressing the systemic, structural components deeply ingrained in societal norms, practices, and institutions that perpetuate unequal treatment and opportunities among genders. Understanding the power dynamics and societal frameworks that fuel gender inequality in diverse contexts is crucial for combating sexism effectively.\n ', 'Sexism is a form of prejudice or discrimination based on a person\'s sex or gender, typically against women but also impacting men. Discrimination can manifest in various ways, including unequal treatment, stereotypes, or demeaning attitudes towards individuals based on their gender. Positive stereotypes, even if seemingly complimentary, can still contribute to sexism by reinforcing harmful beliefs about certain genders. An example of a positive stereotype that is harmful is the belief that women are naturally more nurturing and caring than men, which can limit opportunities for women by prioritizing caregiving roles over personal and professional aspirations. In response to your question about whether the statement that "women are stronger than men since they can sustain labor pains, whose intensity is known to be one of the worst kinds of pain known to humans" is sexist, the emphasis on labor pain as a defining strength can perpetuate stereotypes and reinforce traditional gender roles. It restricts the recognition of other diverse strengths individuals possess, which ultimately contributes to the reinforcement of gender norms and inequality. To make the sentence non-sexist, it could be rephrased to acknowledge various strengths and abilities without attributing them solely to gender, promoting inclusivity and respect for individual differences. If you would like more examples of cases that might not be overtly sexist but still have a sexist undertone, please let me know.']
defi_gen = ["Sexism refers to prejudice, discrimination, or stereotyping based on a person's sex or gender, typically with a bias against one gender over the other. It can manifest in various forms, including unequal treatment, unfair expectations, and harmful attitudes or beliefs about individuals based on their sex. Examples of everyday sexism in the workplace could include paying women less than men for the same job or assuming certain roles are only suitable for one gender. Online sexism could involve making derogatory comments based on gender or perpetuating harmful stereotypes in digital spaces. If you have any more questions or need further clarification, feel free to ask!", "Sexism is a complex issue that involves prejudice, stereotypes, or discrimination against individuals based on their gender. This kind of bias often targets women and embodies the belief that one gender is superior to another. It can take the form of unfair treatment, harmful stereotypes, or limitations on opportunities solely because of a person's gender. Sexism is not limited to individual interactions but can also be ingrained in societal structures, leading to significant adverse effects on both individuals and society.", "Sexism refers to discrimination, prejudice, or stereotyping based on a person's gender. It can manifest in various forms, such as treating individuals differently because of their gender, assuming certain characteristics or behaviors based on gender, or denying opportunities to someone because of their gender. Sexism can affect people of all genders, but it is often directed towards women and can have harmful effects on their lives and opportunities. There are different types of sexism, such as hostile sexism, benevolent sexism, and ambivalent sexism.", "Sexism is a form of discrimination, prejudice, or stereotyping based on a person's gender. It involves attitudes, beliefs, and behaviors that differentiate, marginalize, or treat individuals differently based on their gender. Sexism can manifest in various ways, such as through language, actions, or systemic biases that perpetuate inequality between genders. When analyzing a text for sexism, it's important to assess if it portrays one gender as superior or inferior to the other, reinforces harmful stereotypes, or limits an individual's opportunities or worth based on their gender. This thorough evaluation considers the structure, tone, and phrasing of the text to determine if it perpetuates gender inequality. If you need further assistance or examples, feel free to ask.", 'Certainly! Here\'s an updated prompt that includes the clarification about allowing counter speech:\n\n"Given a piece of text, please analyze and identify any instances of sexism present. Sexism specifically refers to discrimination, prejudice, or stereotyping directed towards individuals or groups based on their gender, where gender is the underlying cause of the abusive behavior. Mentioning the gender of the target alone may not be sufficient for a sentence to be considered sexist. Your task is to help identify and address harmful and unfair attitudes and behaviors towards people because of their gender, while also recognizing that counter speech to challenge these harmful attitudes is permissible."', "I'm glad you found the previous information helpful. I understand you would like a new definition that incorporates the idea that men and women are distinguished based on their social roles and characteristics. Here's an updated definition for you:\n\nHostile sexism refers to overtly negative and hostile attitudes and behaviors towards women based on the belief that women are inferior to men. This can involve open discrimination, derogatory remarks, and devaluation of women. Benevolent sexism, on the other hand, involves attitudes and behaviors that may appear positive or chivalrous on the surface, but actually serve to reinforce traditional gender roles and keep women in subordinate positions. These attitudes often arise from the societal norms and expectations that distinguish men and women based on their perceived social roles and characteristics. Examples of benevolent sexism include patronizing behavior, protective gestures, or idealizing women as pure and delicate. To detect sexism in text, look for language or scenarios that suggest unequal treatment, devaluation, or reinforcement of traditional gender roles. Pay attention to how women are portrayed, whether they are being reduced to stereotypes, denied agency, or discriminated against. Both hostile and benevolent sexism can manifest in various ways in written content, so it's important to analyze the overall context and underlying messages being conveyed. If you have any more questions or need further clarification, feel free to ask!", "Sexism is a form of discrimination, prejudice, or stereotyping based on a person's sex or gender, typically towards women or girls. It involves treating individuals differently or unfairly because of their gender, and can be expressed in various forms, such as unequal treatment, stereotypes, or systemic sexism within societal structures. Let me know if you need more information or have any other questions!", "Certainly! Here's a refined version of the comprehensive definition of sexism that you requested:\n\nSexism is the manifestation of discriminatory, prejudiced, or stereotypical attitudes towards individuals based on their sex or gender. This includes recognizing and addressing the systemic, structural components deeply ingrained in societal norms, practices, and institutions that perpetuate unequal treatment and opportunities among genders. Understanding the power dynamics and societal frameworks that fuel gender inequality in diverse contexts is crucial for combating sexism effectively.", 'Sexism is a form of prejudice or discrimination based on a person\'s sex or gender, typically against women but also impacting men. Discrimination can manifest in various ways, including unequal treatment, stereotypes, or demeaning attitudes towards individuals based on their gender.\n\nPositive stereotypes, even if seemingly complimentary, can still contribute to sexism by reinforcing harmful beliefs about certain genders. An example of a positive stereotype that is harmful is the belief that women are naturally more nurturing and caring than men, which can limit opportunities for women by prioritizing caregiving roles over personal and professional aspirations.\n\nIn response to your question about whether the statement that "women are stronger than men since they can sustain labor pains, whose intensity is known to be one of the worst kinds of pain known to humans" is sexist, the emphasis on labor pain as a defining strength can perpetuate stereotypes and reinforce traditional gender roles. It restricts the recognition of other diverse strengths individuals possess, which ultimately contributes to the reinforcement of gender norms and inequality.\n\nTo make the sentence non-sexist, it could be rephrased to acknowledge various strengths and abilities without attributing them solely to gender, promoting inclusivity and respect for individual differences.\n\nIf you would like more examples of cases that might not be overtly sexist but still have a sexist undertone, please let me know.']
ds = ['CMSB','EDOS','REDDIT','EXIST','HateCheck']
defi_ds = ["Something can be sexist because of its content (what the speaker believes) or because of its phrasing (because of the speaker’s choice of words). For example, the speaker may express sexist attitudes towards gender inequality, either endorsing it or disbelieving it's existence, The speaker may express stereotypes (how genders are traditionally seen and compared to each other) and behavioral expectations (how individuals of a gender should behave according to traditional views). On the other hand, a message may be sexist simply because of how the speaker phrases it–independently from what general beliefs or attitudes the speaker holds. A message is sexist, for example, when it contains attacks, foul language, or derogatory depictions directed towards individuals because of their gender.",'We define sexist content as any abuse, implicit or explicit, that is directed towards women based on their gender, or on the combination of their gender with one or more other identity attributes (e.g. Black women or Muslim women).','For Misogynistic content, we defined four categories: (i) Misogynistic Pejoratives, (ii) descriptions of Misogynistic Treatment, (iii) acts of Misogynistic Derogation and (iv) Gendered Personal attacks against women','Sexism as "prejudice, stereotyping, or discrimination, typically against women, on the basis of sex."','Hate Speech as abuse that is targeted at a protected group or at its members for being a part of that group. We define protected groups based on age, disability, gender identity, familial status, pregnancy, race, national or ethnic origins, religion, sex or sexual orientation, which broadly reflects international legal consensus (particularly the UK’s 2010 Equality Act, the US 1964 Civil Rights Act, and the EU’s Charter of Fundamental Rights).']

In [41]:
def get_cosines(embeddings_dict, emb1, emb2):

    name_1 = None
    name_2 = None
    
    for label, embedding in embeddings_dict.items():
        if np.array_equal(embedding, emb1):
            name_1 = label
        if np.array_equal(embedding, emb2):
            name_2 = label

    if name_1 is None or name_2 is None:
        raise ValueError("Check the embeddings names.")

    emb1_norm = emb1 / np.linalg.norm(emb1, axis=1, keepdims=True)
    emb2_norm = emb2 / np.linalg.norm(emb2, axis=1, keepdims=True)
    
    cosine_matrix = np.dot(emb1_norm, emb2_norm.T)

    return cosine_matrix


In [ ]:
model = SentenceTransformer("all-mpnet-base-v2")

embeddings_participant = model.encode(defi_part)
embeddings_hybrid = model.encode(defi_hybrid)
embeddings_generated = model.encode(defi_gen)
embeddings_ds = model.encode(defi_ds)

embeddings_dict = {
    'Participants': embeddings_participant,
    'Hybrids': embeddings_hybrid,
    'Generated': embeddings_generated,
    'Datasets': embeddings_ds
}

In [43]:
cos_matrix_ds_hu = get_cosines(embeddings_dict, embeddings_ds, embeddings_participant)

cos_matrix_ds_hy= get_cosines(embeddings_dict, embeddings_ds, embeddings_hybrid)

cos_matrix_ds_gen= get_cosines(embeddings_dict, embeddings_ds, embeddings_generated)


In [44]:
type_of_def = 'participant'
CMSB, EDOS, REDDIT, EXIST, HC = split_results(name=type_of_def)
combined_list = [CMSB, EDOS, REDDIT, EXIST, HC]
name_of_lists =['CMSB', 'EDOS', 'REDDIT', 'EXIST', 'HC']


condition_cos_sim = cos_matrix_ds_hu # cos_matrix_ds_hu - cos_matrix_ds_hy - cos_matrix_ds_gen

for nameds,row,ds in zip(name_of_lists,condition_cos_sim,combined_list):
    correlation = np.corrcoef(row, ds)[0, 1]
    print(f"Correlation between cosine similarity performance of {type_of_def} definition on {nameds}")
    print(f"{correlation:.2f}")




Correlation between cosine similarity performance of participant definition on CMSB
-0.49
Correlation between cosine similarity performance of participant definition on EDOS
-0.32
Correlation between cosine similarity performance of participant definition on REDDIT
0.10
Correlation between cosine similarity performance of participant definition on EXIST
-0.09
Correlation between cosine similarity performance of participant definition on HC
-0.71


NOW WITH TFIDF


In [45]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


vectorizer = TfidfVectorizer()

tfidf_participant = vectorizer.fit_transform(defi_part)
tfidf_hybrid = vectorizer.transform(defi_hybrid)
tfidf_generated = vectorizer.transform(defi_gen)
tfidf_ds = vectorizer.transform(defi_ds)

tfidf_dict = {
    'Participants': tfidf_participant,
    'Hybrids': tfidf_hybrid,
    'Generated': tfidf_generated,
    'Datasets': tfidf_ds
}


In [46]:
def get_cosines(tfidf_dict, emb1, emb2):
    name_1 = None
    name_2 = None
    
    for label, tfidf_vector in tfidf_dict.items():
        if np.array_equal(tfidf_vector.toarray(), emb1.toarray()):
            name_1 = label
        if np.array_equal(tfidf_vector.toarray(), emb2.toarray()):
            name_2 = label

    if name_1 is None or name_2 is None:
        raise ValueError("Check the embeddings names.")
    
    cos_matrix = cosine_similarity(emb1, emb2)

    return cos_matrix

In [47]:
cos_matrix_ds_hu_tfidf = get_cosines(tfidf_dict, tfidf_ds, tfidf_participant)
cos_matrix_ds_hy_tfidf = get_cosines(tfidf_dict, tfidf_ds, tfidf_hybrid)
cos_matrix_ds_gen_tfidf = get_cosines(tfidf_dict, tfidf_ds, tfidf_generated)


In [48]:
type_of_def = 'GPT'
CMSB, EDOS, REDDIT, EXIST, HC = split_results(name=type_of_def)
combined_list = [CMSB, EDOS, REDDIT, EXIST, HC]
name_of_lists =['CMSB', 'EDOS', 'REDDIT', 'EXIST', 'HC']
 
condition_tfidf = cos_matrix_ds_hu_tfidf # cos_matrix_ds_hu_tfidf - cos_matrix_ds_hy_tfidf - cos_matrix_ds_gen_tfidf

for nameds,row,ds in zip(name_of_lists,condition_tfidf,combined_list):
    correlation = np.corrcoef(row, ds)[0, 1]
    print(f"Correlation between cosine similarity performance of {type_of_def} definition on {nameds}")
    print(f"{correlation:.2f}")


Correlation between cosine similarity performance of GPT definition on CMSB
-0.42
Correlation between cosine similarity performance of GPT definition on EDOS
-0.01
Correlation between cosine similarity performance of GPT definition on REDDIT
0.63
Correlation between cosine similarity performance of GPT definition on EXIST
-0.62
Correlation between cosine similarity performance of GPT definition on HC
-0.14
